# DigiFab Datathon

Aim: Centroid Distances

In [11]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error 
from sklearn import *
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

## Imports

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
file_path = '/content/drive/MyDrive/DigiFab_Datathon/train_and_test_sets/'
save_path = '/content/drive/MyDrive/DigiFab_Datathon/'

train_descriptors = pd.read_csv(file_path+"train_descriptors.csv")
train_mord3d = pd.read_csv(file_path+"train_mord3d.csv")
train_morgan = pd.read_csv(file_path+"train_morgan.csv")
train_rdk = pd.read_csv(file_path+"train_rdk.csv")

train_crystals = pd.read_csv(file_path+"train_crystals.csv")
train_distances = pd.read_csv(file_path+"train_distances.csv")
train_centroid_distances = pd.read_csv(file_path+"train_centroid_distances.csv")

In [7]:
test_descriptors = pd.read_csv(file_path+"test_descriptors.csv")
test_mord3d = pd.read_csv(file_path+"test_mord3d.csv")
test_morgan = pd.read_csv(file_path+"test_morgan.csv")
test_rdk = pd.read_csv(file_path+"test_rdk.csv")

## Date Engineering Step

* Look for NaN values.
* Look at magintude of values - do they need scaling?

In [ ]:
print(train_descriptors.describe())
print(train_mord3d.describe())

         Unnamed: 0           ABC  ...      mZagreb1      mZagreb2
count  13449.000000  13449.000000  ...  13448.000000  13449.000000
mean   40464.558183      8.791685  ...      4.820664      2.709742
std    19446.921704      2.294386  ...      1.421185      0.606246
min        2.000000      0.000000  ...      0.750000      0.000000
25%    33281.000000      7.387307  ...      3.833333      2.333333
50%    43815.000000      9.289847  ...      4.722222      2.833333
75%    55711.000000     10.663621  ...      5.694444      3.194444
max    66123.000000     13.654635  ...     10.652778      4.000000

[8 rows x 1614 columns]
         Unnamed: 0         PNSA1  ...        MOMI-Z           PBF
count  13449.000000  13432.000000  ...  13449.000000  13449.000000
mean   40464.558183    164.398549  ...    356.137203      0.383000
std    19446.921704     55.235914  ...    284.378542      0.304775
min        2.000000      0.000000  ...      0.000000      0.000000
25%    33281.000000    127.855661  ..

In [ ]:
# Data Preprocessing

In [8]:
train_mord3d_full = train_mord3d.iloc[:, 1:-3].dropna(axis= 1, how="any")
train_mord3d_full.shape

(13449, 173)

In [9]:
test_mord3d_full = test_mord3d[train_mord3d_full.columns]
test_mord3d_full.shape

(3363, 173)

In [10]:
train_PCA = decomposition.PCA(n_components=.95)
scaler_for_PCA = preprocessing.StandardScaler()
train_mord3d_PCA = train_PCA.fit_transform(scaler_for_PCA.fit_transform(train_mord3d_full))
test_mord3d_PCA = train_PCA.transform(scaler_for_PCA.transform(test_mord3d_full))
print(train_mord3d_PCA.shape, test_mord3d_PCA.shape)

(13449, 45) (3363, 45)


In [12]:
train_crystals["is_centrosymmetric"] = train_crystals["is_centrosymmetric"].astype(int)

## Initial Models

In [25]:
#RandomForest

In [26]:
target = "is_centrosymmetric"

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
clf = RandomForestClassifier(n_estimators = 100)

In [29]:
clf = RandomForestClassifier(n_estimators = 100)
clf.fit(train_mord3d_PCA,train_crystals[target])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [30]:
y_pred = clf.predict(train_mord3d_PCA)

In [ ]:
#Cross-validation

In [31]:
from sklearn.model_selection import cross_val_score

In [32]:
np.mean(cross_val_score(clf, train_mord3d_PCA, train_crystals[target], cv=10))

0.848762502212781

### Outputting predictions for automated checking 
Link: https://github.com/stevenkbennett/fons_datathon_testing

In [33]:
predictions = logreg.predict(test_mord3d_PCA)

In [34]:
# np.savetxt("is_centrosymmetric.csv", predictions)
# Alternatively, if using pandas, you can use the following:
pd.DataFrame(predictions).to_csv(save_path+"task_2_predictions.csv", header=False, index=False)